In [ ]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from keras.models import load_model

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=32, epochs=100, validation_data=(X_val_fold, Y_val_fold))

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "models/final_model.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.2f}')
else:
    print("No folds to calculate average accuracy.")

In [22]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from keras.models import load_model

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/dataset_restructure",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

print('X_test:', X_test)
print('Y_test:', Y_test)


Found 960 files belonging to 60 classes.
X_test: [[[[ 41.29586   29.091717  29.886095]
   [144.56955  114.14943   99.56807 ]
   [ 42.90385   36.201187  30.232252]
   ...
   [249.36412  228.76942  197.04895 ]
   [222.92735  186.34895  140.19806 ]
   [ 33.603527  26.911217  47.974834]]

  [[ 63.45859   42.53699   31.069523]
   [245.26923  227.65384  203.79437 ]
   [ 62.446747  38.477818  31.977818]
   ...
   [ 52.930622  38.40697   33.39951 ]
   [ 46.372757  34.128662  28.739597]
   [255.       228.57693  207.27077 ]]

  [[164.85947  139.8491    99.71597 ]
   [193.5444   166.78258  148.2589  ]
   [150.78699  118.571     90.056206]
   ...
   [201.10146  151.21681   99.84418 ]
   [ 36.985256  36.023705  39.239677]
   [215.48961  194.88457  174.0976  ]]

  ...

  [[154.05728  133.46277  109.06789 ]
   [105.920364  84.75188   60.643982]
   [246.08147  224.88914  201.3639  ]
   ...
   [255.       228.88464  192.2174  ]
   [201.35637  160.72617  109.69535 ]
   [242.80376  203.72989  166.0479  

2024-05-03 10:26:50.551777: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
import tensorflow as tf
from keras.models import load_model
new_model = tf.keras.models.load_model('models/final_model.h5')
# Show the model architecture
new_model.summary()

Model: "functional_49"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 52, 52, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 58, 58, 3) │          0 │ input_layer_24[0… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 26, 26,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 26, 26,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 26, 26,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 28, 28,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 13, 13,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 13, 13,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 13, 13,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 13, 13,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 13, 13,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 13, 13,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 13, 13,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 13, 13,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 13, 13,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 13, 13,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 13, 13,    │      1,024 │ conv2_block1_3_c

 Total params: 24,644,158 (94.01 MB)

 Trainable params: 24,591,036 (93.81 MB)

 Non-trainable params: 53,120 (207.50 KB)

 Optimizer params: 2 (12.00 B)

In [9]:

predictions = new_model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step


In [20]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming `predictions` is the output of your model and `Y_test` is your true labels
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(Y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_classes, predicted_classes)
print(f'Accuracy: {accuracy:.4f}')  # Modified to show 4 decimal places

# Complete classification report
print(classification_report(true_classes, predicted_classes, digits=3))

Accuracy: 0.9948
              precision    recall  f1-score   support

           0      1.000     1.000     1.000         6
           1      1.000     1.000     1.000         5
           2      1.000     1.000     1.000         2
           3      1.000     1.000     1.000         5
           4      1.000     1.000     1.000         2
           5      1.000     1.000     1.000         6
           6      1.000     1.000     1.000         5
           7      1.000     1.000     1.000         2
           9      1.000     1.000     1.000         2
          11      1.000     1.000     1.000         2
          12      1.000     1.000     1.000         3
          13      1.000     1.000     1.000         5
          14      1.000     1.000     1.000         4
          15      1.000     1.000     1.000         2
          16      1.000     1.000     1.000         3
          17      1.000     1.000     1.000         3
          18      1.000     1.000     1.000         4
          